# 뉴스 요약문 자동 평가 지표 테스트

- ROUGE, BLEU 등 대표적인 요약 평가 지표로 결과를 평가합니다.
- 원본: kobert_full_news_summaries_final.csv (또는 kobert_full_news_summaries.csv)
- 요약문: 'KoBERT_요약' 컬럼
- 기준문: '원본_내용' 또는 별도 정답 요약문이 있을 경우 해당 컬럼 사용
- 한글 뉴스 데이터에 적합한 평가 방식 안내 포함

In [2]:
# 필수 패키지 설치 (최초 1회만)
%pip install rouge-score nltk


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
# =============================
# SBERT 설치 및 임포트
# =============================
%pip install -q sentence-transformers


Note: you may need to restart the kernel to use updated packages.
ROUGE 계산 중...
BLEU 계산 중...
SBERT 유사도 계산 중...


OSError: snunlp/KR-SBERT-V40K-klueNLI-aug is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [3]:

import pandas as pd
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 평가 대상 데이터 로드
df = pd.read_csv('./data/kobert_full_news_summaries_final.csv')
print(f'총 샘플 수: {len(df):,}개')
print(df.columns)
df = df.dropna(subset=['원본_내용', 'KoBERT_요약'])
print(f'유효 샘플 수: {len(df):,}개')

총 샘플 수: 10,812개
Index(['기사_번호', '원본_제목', '원본_내용', 'KoBERT_요약', '원본_길이', '요약_길이', '압축률(%)'], dtype='object')
유효 샘플 수: 10,812개


In [5]:
# ROUGE 평가 함수
def compute_rouge(refs, hyps):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for ref, hyp in zip(refs, hyps):
        score = scorer.score(str(ref), str(hyp))
        for k in scores.keys():
            scores[k].append(score[k].fmeasure)
    return {k: np.mean(v) for k, v in scores.items()}

In [ ]:
# SBERT 기반 요약문 평가 (Cosine Similarity)
from sentence_transformers import SentenceTransformer, util  # SBERT 임포트
import pandas as pd  # 데이터프레임 처리
import numpy as np  # 수치 연산용

# 1. 평가 대상 데이터 로드
df = pd.read_csv('./data/kobert_full_news_summaries_final.csv')  # 데이터 파일 읽기
df = df.dropna(subset=['원본_내용', 'KoBERT_요약'])  # 결측치 제거
refs = df['원본_내용'].astype(str).tolist()  # 원본 문장 리스트
hyps = df['KoBERT_요약'].astype(str).tolist()  # 요약문 리스트

# 2. SBERT 임베딩 기반 유사도 평가 함수 정의
def compute_sbert_similarity(refs, hyps, model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
    """
    두 문장 리스트(refs, hyps)에 대해 SBERT 임베딩을 구하고,
    코사인 유사도의 평균값을 반환합니다.
    """
    model = SentenceTransformer(model_name)  # 사전학습된 SBERT 모델 로드
    ref_emb = model.encode(list(refs), convert_to_tensor=True, show_progress_bar=True)  # 원본 임베딩
    hyp_emb = model.encode(list(hyps), convert_to_tensor=True, show_progress_bar=True)  # 요약문 임베딩
    sims = util.cos_sim(ref_emb, hyp_emb).diagonal().cpu().numpy()  # 각 쌍의 코사인 유사도
    return float(np.mean(sims))  # 평균 유사도 반환

# 3. 평가 실행 및 결과 출력
print('SBERT 유사도 계산 중...')
sbert_result = compute_sbert_similarity(refs, hyps)

print('\n===== 요약문 SBERT 평가 결과 =====')
print(f"SBERT Cosine Similarity: {sbert_result:.4f}")
print('\n(1에 가까울수록 두 문장이 의미적으로 유사함을 의미합니다.)')

ROUGE 계산 중...
BLEU 계산 중...
BLEU 계산 중...
SBERT 유사도 계산 중...
SBERT 유사도 계산 중...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/338 [00:00<?, ?it/s]

Batches:   0%|          | 0/338 [00:00<?, ?it/s]


===== 요약문 자동 평가 결과 =====
ROUGE-1: 0.4766
ROUGE-2: 0.2159
ROUGE-L: 0.4642
BLEU: 0.3263
SBERT Cosine Similarity: 0.8569
